# Stimulus Response Analyses

This notebook explores stimulus response analysis to stimuli & events of interest.

Investigated events:



In [1]:
%config Completer.use_jedi = False

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
from matplotlib.lines import Line2D

from convnwb.io import load_nwbfile

from spiketools.plts.trials import plot_rate_by_time
from spiketools.plts.annotate import add_vlines
from spiketools.plts.style import drop_spines
from spiketools.plts.trials import plot_rasters, create_raster_title
from spiketools.plts.utils import make_axes, save_figure, make_grid, get_grid_subplot
from spiketools.utils.epoch import epoch_spikes_by_event
from spiketools.utils.base import select_from_list
from spiketools.measures.trials import compute_trial_frs
from spiketools.stats.trials import compare_pre_post_activity

In [3]:
import seaborn as sns
sns.set_context('talk')

In [4]:
import sys
sys.path.append('../code')
#from plts import plot_task_structure
from utils import group_array_by_key

## Settings

In [5]:
# Define the base data folder
data_folder = '/Users/weijiazhang/Data/Train_NWB'
# Define subject information
session = {
    'experiment' : 'TRAIN',
    'subject' : 'R1219C',
    'session' : 0    
}

In [6]:
# Set a unit of interest to select
uind = 2

In [7]:
# Set the time range to analyze, and pre & post trial windows (in seconds)
trial_range = [-2, 2]
pre_window = [-2, 0]
post_window = [0, 2]

## Load Data

In [8]:
# Load NWB file
nwbfile = load_nwbfile(session, data_folder)

In [9]:
nwbfile.trials

trials pynwb.epoch.TimeIntervals at 0x5095253712
Fields:
  colnames: ['start_time' 'stop_time' 'drive_type' 'feedback_start_time'
 'feedback_stop_time' 'fixation_start_time' 'fixation_stop_time'
 'hold_start_time' 'hold_stop_time' 'movement_start_time'
 'movement_stop_time' 'object' 'object_position' 'response_error'
 'response_position' 'response_time' 'trial_type']
  columns: (
    start_time <class 'hdmf.common.table.VectorData'>,
    stop_time <class 'hdmf.common.table.VectorData'>,
    drive_type <class 'hdmf.common.table.VectorData'>,
    feedback_start_time <class 'hdmf.common.table.VectorData'>,
    feedback_stop_time <class 'hdmf.common.table.VectorData'>,
    fixation_start_time <class 'hdmf.common.table.VectorData'>,
    fixation_stop_time <class 'hdmf.common.table.VectorData'>,
    hold_start_time <class 'hdmf.common.table.VectorData'>,
    hold_stop_time <class 'hdmf.common.table.VectorData'>,
    movement_start_time <class 'hdmf.common.table.VectorData'>,
    movement_sto

In [10]:
# Get object and object position
nwbfile.trials
obj = nwbfile.trials.object[:]
obj_pos = nwbfile.trials.object[:]
# res_pos = nwbfile.trials.response_position[:]
# res_time = nwbfile.trials.response_time[:]



In [11]:
# Check the number of units 
n_units = len(nwbfile.units)

print('Number of units: {}'.format(n_units))
#print('Number to keep:  {}'.format(n_keep))

Number of units: 23


In [12]:
# Get spikes for unit of interest
spikes = nwbfile.units.get_unit_spike_times(uind)

## Subject response time and position

In [13]:
# Get subject response position and time 

res_pos = nwbfile.trials.response_position[:]
res_time = nwbfile.trials.response_time[:]

In [14]:
# Check the number of responses

n_response = len(res_time)
print('Number of responses: {}'.format(n_response ))

Number of responses: 64


In [15]:
# Gather neural data around trials of interest
all_trials_response = epoch_spikes_by_event(spikes, res_time, trial_range)

In [17]:
# Compute averages & statistical comparison of pre / post window firing rates
avg_pre, avg_post, t_val, p_val = compare_pre_post_activity(all_trials_response, pre_window, post_window)

## Subject response - Split by Stimulus Content

In [32]:
# Create masks to sub-select objects with respect to response time
mask_res_time = group_array_by_key(obj, res_time)
mask_res_time
barrel_res_time =  np.array(mask_res_time[b'barrel'][:])
box_res_time =  np.array(mask_res_time[b'box'][:])
bench_res_time =  np.array(mask_res_time[b'bench'][:])
desk_res_time =  np.array(mask_res_time[b'desk'][:])


num_barrel = len(barrel_res_time)

num_box = len(box_res_time)

num_bench = len(bench_res_time)

num_desk= len(desk_res_time)

In [35]:
# Check the number of trials with respect to each individual objects
print('Number of barrel trials: \t{}'.format(num_barrel))
print('Number of box trials: \t{}'.format(num_box))
print('Number of bench trials: \t{}'.format(num_bench))
print('Number of desk trials: \t{}'.format(num_desk))


Number of barrel trials: 	16
Number of box trials: 	16
Number of bench trials: 	16
Number of desk trials: 	16
